<a href="https://colab.research.google.com/github/raymondwcs/learning_spacy/blob/main/iDDY_dataset_Threads_(Top_50_tokens%2C_nouns%2C_verbs%2C_adjectives_and_named_entities).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet sentence-transformers
!pip install --upgrade spacy[cuda102]
!pip install -U spacy_pkuseg
!python -m spacy download zh_core_web_lg

     |████████████████████████████████| 603.8 MB 8.8 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_lg')


In [ ]:
# from sentence_transformers import SentenceTransformer
# from sklearn.cluster import KMeans
# from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import numpy as np
import pandas as pd
import spacy

In [ ]:
!gdown --id 1bw2rAvh8XO80YIg1qr5dZGLf35gBanVR

Downloading...
From: https://drive.google.com/uc?id=1bw2rAvh8XO80YIg1qr5dZGLf35gBanVR
To: /content/thread.tsv
49.7MB [00:00, 120MB/s] 


In [ ]:
CHECKPOINT = 'paraphrase-multilingual-mpnet-base-v2'
# CHECKPOINT = 'paraphrase-multilingual-mpnet-base-v2'

dataset = pd.read_csv('thread.tsv',sep='\t\t',header=None,engine='python')
print(dataset.shape)   

subjects = []
for s in dataset.iloc[:,9]:
    s = str(s)
    subjects.append(s.replace(' ','')) # remove extra spaces

del dataset

(215845, 18)


In [ ]:
tk = []       # tokens
noun = []     # nouns, proper nouns
verb = []     # verbs
adj = []      # adjectives
ne = []       # named entities

print(spacy.prefer_gpu()) # or spacy.require_gpu()
nlp = spacy.load('zh_core_web_lg')
# Use trained pkuseg model on Traditional Chinese text
# Download folder "models" from:
# https://github.com/raymondwcs/learning_spacy/tree/main/spacy_pkuseg
nlp.tokenizer.initialize(pkuseg_model='./models')

from spacy.lang.zh.stop_words import STOP_WORDS
spacy_stopwords = STOP_WORDS
for w in spacy_stopwords:
  nlp.vocab[w].is_stop = True

for subject in tqdm(subjects, leave=True):
    doc = nlp(subject)
    # remove stop words
    tokens = [token for token in doc if not token.is_stop]
    # for token in doc:
    for token in tokens:
        tk.append(token.text)
        if token.pos_ == 'NOUN' or token.pos_ == 'PROPN':
            noun.append(token.text)
        elif token.pos_ == 'VERB':
            verb.append(token.text)
        elif token.pos_ == 'ADJ':
            adj.append(token.text)
    for ent in doc.ents:
        ne.append('(%s, %s)' % (ent.text, ent.label_))

False


100%|██████████| 215845/215845 [30:26<00:00, 118.16it/s]


In [ ]:
pd.Series(tk).value_counts().head(50)

點      11837
今日     10717
跌       9650
買       9512
升       9044
個       8950
係       8723
會       7907
!!      7555
中國      6522
香港      6501
美國      5734
港股      5593
幾       4642
股票      4155
過       4149
美股      4067
隻       4024
騰訊      3735
大戶      3668
牛       3644
恆指      3621
股市      3460
。。。     3417
想       3406
明天      3157
未       3101
將       3038
700     2902
爆升      2856
點解      2854
夜期      2824
話       2813
做       2784
睇       2718
「       2656
中       2561
無       2559
先       2549
冇       2547
股災      2454
已經      2454
年       2441
聽日      2423
蚊       2356
投資      2355
開始      2348
新       2347
星期一     2346
錢       2301
dtype: int64

In [ ]:
pd.Series(noun).value_counts().head(50)

今日     10717
中國      6522
香港      6501
美國      5734
港股      5593
股票      4155
美股      4067
騰訊      3570
牛       3497
股市      3460
大戶      3202
明天      3157
夜期      2819
星期一     2346
聽日      2276
小米      2270
消息      2237
錢       2193
經濟      2098
大陸      2094
??      1977
指數      1898
日       1888
港       1885
今晚      1866
左       1827
月       1806
銀行      1792
今次      1698
匯豐      1585
QQ      1566
熊       1512
淡友      1511
大師      1477
全球      1466
股災      1462
大市      1441
星期      1438
金融      1430
業績      1423
阿爺      1393
公司      1370
恒指      1325
老美      1306
股價      1271
日日      1261
市場      1252
收市      1233
中美      1231
美       1222
dtype: int64

In [ ]:
pd.Series(verb).value_counts().head(50)

跌     9598
買     9280
升     8643
係     7630
想     3406
過     2820
做     2784
睇     2711
爆升    2693
指     2133
入     2079
走     1963
玩     1884
嘩     1813
完     1781
見     1716
賺     1701
有無    1630
大跌    1598
會     1575
投資    1539
插     1531
準備    1467
講     1451
無     1437
為     1413
小心    1334
問     1331
輸     1291
炒     1289
穿     1271
反彈    1260
死     1259
係咪    1256
冇人    1250
咩事    1197
成     1192
出現    1145
恆指    1103
上市    1098
聽     1054
爆     1048
沽     1041
冇     1038
大升     965
來      928
返      916
知      913
分析     909
炒股     885
dtype: int64

In [ ]:
pd.Series(adj).value_counts().head(50)

新      1973
恆指      639
高       590
壞       398
個       395
低       306
幾多      294
原來      283
無       263
前       249
倒跌      243
突發      239
窮       230
沽空      224
未來      223
超級      217
內地      212
標普      188
股災      179
溫馨      177
財經      167
假       144
大戶      136
強       134
利好      133
女       132
死       132
差       118
副       116
最低      114
邊間      107
相關      106
大型      106
斷估      105
唯一      101
真       101
會        96
內        95
道期       95
正        94
這個       93
仆街       89
超大       88
人民幣      87
人工       86
尋日       86
嘅        85
有乜       85
開始       85
倒升       85
dtype: int64

In [ ]:
pd.Series(ne).value_counts().head(50)

(今日, DATE)           7003
(香港, GPE)            5713
(美國, GPE)            4725
(中國, GPE)            3949
(港股, GPE)            3425
(明天, DATE)           3025
(騰訊, ORG)            2517
(美股, GPE)            2440
(700, CARDINAL)      1908
(星期一, DATE)          1876
(今晚, TIME)           1749
(港, GPE)             1723
(大陸, GPE)            1597
(三, CARDINAL)        1453
(小米, ORG)            1400
(一, CARDINAL)        1216
(3, CARDINAL)        1163
(今年, DATE)           1094
(今天, DATE)           1052
(阿里, ORG)             954
(下午, TIME)            942
(美, GPE)              876
(5, CARDINAL)         871
(2, CARDINAL)         781
(第一, ORDINAL)         752
(下星期, DATE)           748
(1, CARDINAL)         700
(中美, ORG)             663
(日, GPE)              648
(10, CARDINAL)        587
(沈, PERSON)           581
(老美, GPE)             579
(9, CARDINAL)         578
(巴巴, PERSON)          569
(兩, CARDINAL)         569
(港交所, ORG)            567
(美團, ORG)             552
(4, CARDINAL)         552
(夜期, PERSON)